# Analyzing Traffic Patterns in Austin, Texas
Austin, the capital city of Texas, has become a hub for rapid population growth, economic expansion, and increased transportation demands. As the city expands, managing traffic effectively becomes critical in maintaining quality of life, ensuring smooth mobility, and supporting public safety.

## Project and Data Overview
Our objective is to analyze traffic data collected from several major road intersections in Downtown and Central Austin—**Burnet**, **Congress**, **Lamar**, and **Loop 360**—and leverage machine learning techniques to predict traffic levels. By identifying traffic patterns and trends over time, we aim to provide insights for enhancing urban mobility and reducing congestion in Austin.

<img src="austinDataMap.jpg" width="450">

### Dataset
The dataset, provided by the [City of Austin](https://data.austintexas.gov/Transportation-and-Mobility/Radar-Traffic-Counts/i626-g7ub/about_data), contains traffic volume and speed data collected using Wavetronix radar sensors from June 18, 2017, to September 9, 2021. The data captures volume data per intersection per direction in 15 minute bins, with additional temporal features.

The data focuses on a small subset of intersections but provides rich granularity, making it suitable for uncovering actionable patterns. Additionally, it is important to note that this dataset is historical and no longer updated, as the Wavetronix radar sensors were phased out in 2021.

### Columns
Here is a description of the columns in the dataset:
- **`Row ID`**: A unique identifier for each row, created as a hash of `Row ID`, `Detector ID`, and `Intersection Name`.
- **`Detector ID`**: A unique number given to each traffic count lane monitored by a radar sensor.
- **`KITS ID`**: The ID of the Wavetronix sensor in the city’s Advanced Traffic Management System (KITS).
- **`Read Date`**: The timestamp when the radar device collected data.
- **`Intersection Name`**: Name of the intersection or general location where the sensor is deployed.
- **`Lane`**: Direction and specific lane monitored (e.g., NB-Inside Lane or SB-Outside Lane).
- **`Volume`**: Number of vehicles detected in the last 15 minutes.
- **`Occupancy`**: The percentage of time a vehicle was detected in the sensor's field of view in the 15-minute period.
- **`Speed`**: Average speed of vehicles in the 15-minute period (in mph).
- **`Month`**: Numeric representation of the month (e.g., January = 1).
- **`Day`**: Numeric representation of the day of the month.
- **`Year`**: Year of the data record.
- **`Hour`**: Hour of the day when the data was collected (24-hour format).
- **`Minute`**: Minute of the hour when the data was collected.
- **`Day of Week`**: Day of the week, represented as a number (Sunday = 0, Monday = 1, etc.).
- **`Time Bin`**: Text representation of the 15-minute time interval.
- **`Direction`**: Traffic direction of the monitored lane (e.g., NB = Northbound, SB = Southbound).

*Before we start the analysis, let’s prepare the dataset for exploration.*

## Import Libraries & Read Data

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

#load data
df = pd.read_csv('Radar_Traffic_Counts_20241114.csv')
df.head()

,Row ID,Detector ID,KITS ID,Read Date,Intersection Name,Lane,Volume,Occupancy,Speed,Month,Day,Year,Hour,Minute,Day of Week,Time Bin,Direction
0,9794d3655e0572eb865546d0df50e00f,74,19,01/24/2018 04:15:01 AM,KINNEYLAMAR,Lane1,4,0,18,1,23,2018,22,15,2,22:15,NaN
1,1f51f4e6a68297c9f9907e4361313c47,92,24,12/17/2017 01:45:00 AM,LOOP 360LAKEWOOD,NB_out,103,6,40,12,16,2017,19,45,6,19:45,NB
2,9de27c4292306d305fc781dad25347c4,10,3,01/01/156489 12:00:00 AM,LAMARSHOALCREEK,SB_out,20,1,33,8,3,2019,23,15,6,23:15,SB
3,b5a5739acbc9481c9bdb8854d31787c9,1,1,04/03/2020 11:15:00 PM,LAMARMANCHACA,NB_out,8,0,32,4,3,2020,23,15,5,23:15,NB
4,8b4b5e8a088d964b32ea95f36b146ea4,42,14,01/24/2018 03:45:01 AM,Robert E LeeBarton Springs,SB,44,2,27,1,23,2018,21,45,2,21:45,SB


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6834237 entries, 0 to 6834236
Data columns (total 17 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Row ID             object
 1   Detector ID        int64 
 2   KITS ID            int64 
 3   Read Date          object
 4   Intersection Name  object
 5   Lane               object
 6   Volume             int64 
 7   Occupancy          int64 
 8   Speed              int64 
 9   Month              int64 
 10  Day                int64 
 11  Year               int64 
 12  Hour               int64 
 13  Minute             int64 
 14  Day of Week        int64 
 15  Time Bin           object
 16  Direction          object
dtypes: int64(11), object(6)
memory usage: 886.4+ MB


In [5]:
#count of null values for each column
df.isnull().sum()

Row ID                    0
Detector ID               0
KITS ID                   0
Read Date                 0
Intersection Name         0
Lane                      0
Volume                    0
Occupancy                 0
Speed                     0
Month                     0
Day                       0
Year                      0
Hour                      0
Minute                    0
Day of Week               0
Time Bin                  0
Direction            401274
dtype: int64

### Initial Notes:
* The dataset contains 6.83M rows and 17 columns.
* The `Direction` column contains 401,274 null values, approximately 5.87% of the dataset, posing a problem to data integrity.

## Data Processing
In this section, we process the dataset to prepare it for analysis and modeling. The steps include:
- Dropping unrelated columns to reduce complexity.
- Handling missing values and filtering data to focus on relevant features.
- Grouping traffic data by hour for each intersection and direction, calculating key metrics.
- Engineering features such as a `Full Date` column and a `Holiday` indicator.

### Dropping Unrelated Columns
The dataset includes several columns that are repetitive, unhelpful, or add unnecessary complexity for our project goals. These columns include:
- **`Row ID`**, **`Detector ID`, and `KITS ID`**: IDs for each row and traffic monitoring devices, offering no additional insight.
- **`Read Date`**: We ran into problems sorting the data referring to this column, as some read dates were clocked out of chronological order due to the date time data type recording exact details (e.g. `2021-09-07 23:00:00` versus `2021-09-07 23:00:01`). The dataset already provides temporal information in separate columns (`Year`, `Month`, `Day`, `Hour`, `Minute`), so we drop this column.
- **`Lane`**: Detailed lane information is too granular for this project, though of interest in the future.
- **`Occupancy`**: We wanted to include this column, but the data card did not provide a helpful description, plus the range of values did not align with external definitions of this feature.
- **`Minute`**: Dropped since analysis focuses on hourly data.
- **`Time Bin`**: Textual representation of time, redundant due to numerical time columns.

By dropping these columns, we streamline the dataset and focus on meaningful variables for traffic analysis.

In [9]:
# drop unrelated columns
df = df.drop(columns=['Row ID', 'Detector ID', 'KITS ID', 'Read Date', 'Occupancy', 'Lane', 'Minute', 'Time Bin'])

### Handling Missing Values
To handles issues particularly in the `Direction` column, we:
1. Remove rows with any `NaN` values.
2. Filter the dataset to include only rows where `Direction` is **NB (Northbound)** or **SB (Southbound)**. 
   - This decision simplifies the analysis and focuses on primary directions (due to limits on computational power). So while Eastbound and Westbound directions are of interest, their inclusion is deferred for future work.

In [11]:
#drop nulls
df = df.dropna()
df = df[df.astype(str).apply(lambda row: 'NaN' not in row.values, axis=1)]

#filter out rows whose 'Direction' is not 'NB' or 'SB'
df = df[df['Direction'].isin(['NB', 'SB'])]

### Adjust Intersection Names

In [13]:
df['Intersection Name'].value_counts()

Intersection Name
LOOP 360LAKEWOOD              651342
LOOP 360WALSH TARLTON         643972
BURNETPALM WAY                515405
BurnetRutland                 497380
LAMARMANCHACA                 437900
LAMARSHOALCREEK               427132
LAMARCOLLIER                  390660
CONGRESSBARTON SPRINGS        387083
LAMARSANDRA MURAIDA           313100
CongressJohanna               249248
N Lamar15th                   220692
LAMARZENNIA                   202620
LOOP 360CEDAR                 182920
Robert E LeeBarton Springs    111954
Name: count, dtype: int64

The major intersections include Loop 360, Cesar Chavez, Burnet, Lamar, Congress, and Barton Springs. We'd like to keep the granular level of these intersections instead of mapping to common roads.

Also, in the time that this data was collected, one intersection `Robert E LeeBarton Springs` was replaced with `Azie Morton Barton Springs`. However, the dataset did not get automatically updated, so we adjust that single intersection name to reflect the change.

In [15]:
#correct outdated intersection name
df['Intersection Name'] = df['Intersection Name'].replace(
    {'Robert E LeeBarton Springs': 'Azie MortonBarton Springs'})

In [16]:
df.head()

,Intersection Name,Volume,Speed,Month,Day,Year,Hour,Day of Week,Direction
1,LOOP 360LAKEWOOD,103,40,12,16,2017,19,6,NB
2,LAMARSHOALCREEK,20,33,8,3,2019,23,6,SB
3,LAMARMANCHACA,8,32,4,3,2020,23,5,NB
4,Azie MortonBarton Springs,44,27,1,23,2018,21,2,SB
5,Azie MortonBarton Springs,13,28,1,23,2018,21,2,NB


### Feature Engineering & Sorting
We create a new `Full Date` column by combining the `Year`, `Month`, `Day`, and `Hour` columns and setting it to the date time data type. This feature allows us to analyze traffic trends over time as well as sort the data accordingly.

Upon closer inspection, we observed:
- The date **`2021-09-08`** contains data only up to hour 12 (half a day).
- The date **`2021-09-09`** has sparse rows due to the dataset being phased out after September 9, 2021.

To maintain data integrity, rows with dates on or after **`2021-09-08`** are removed. The dataset is then sorted by `Full Date` in descending order to ensure the most recent data appears first.

In [18]:
#create 'Full Date' column and drop rows after '2021-09-08'
df['Full Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
df = df[df['Full Date'] < '2021-09-08']

#sort dataset
df = df.sort_values(by='Full Date', ascending=False).reset_index(drop=True)

### Grouping and Aggregating Traffic Data
For any given day, we wanted to see the average speed and total volume of traffic by the hour, broken down by the available columns. This allows us to analyze hourly traffic trends and identify patterns across different roads and time periods.

To achieve this, for each combination of intersection, date, hour, and direction we calculate:
1. **Average Speed**: The mean speed of vehicles, rounded to the nearest whole number.
2. **Total Volume**: The sum of vehicles detected.

To account for missing combinations (e.g., hours with no data), we generate a full cartesian product of all possible combinations of:
- `Intersection Name`
- `Year`, `Month`, `Day`
- `Hour`
- `Direction`

Missing values for `Volume` are filled with **0**, while missing `Speed` values remain as `NaN` to avoid introducing bias.

In [20]:
#combos of all possible vals for Intersection Name, Date, Hour, & Direction
unique_dates = df[['Year', 'Month', 'Day', 'Day of Week']].drop_duplicates()
unique_road_names = df['Intersection Name'].unique()
unique_hours = pd.DataFrame({'Hour': np.arange(24)})
unique_directions = pd.DataFrame({'Direction': ['NB', 'SB']})

#generate full cartesian product
cartesian = (
    unique_dates
    .merge(pd.DataFrame({'Intersection Name': unique_road_names}), how='cross')
    .merge(unique_hours, how='cross')
    .merge(unique_directions, how='cross'))

#calc grouped stats, now including occupancy
grouped = (
    df.groupby(['Intersection Name', 'Year', 'Month', 'Day', 'Hour', 'Direction'])
    .agg(
        Speed=('Speed', lambda x: int(round(x.mean()))),
        Volume=('Volume', 'sum')).reset_index())

#calc grouped stats
grouped = (
    df.groupby(['Intersection Name', 'Year', 'Month', 'Day', 'Hour', 'Direction'])
    .agg(
        Speed=('Speed', lambda x: int(round(x.mean()))),
        Volume=('Volume', 'sum')).reset_index())

#merge grouped stats w full cartesian product to fill in missing combos
result = cartesian.merge(
    grouped,
    on=['Intersection Name', 'Year', 'Month', 'Day', 'Hour', 'Direction'],
    how='left')

#missing Speed = NaN, missing Volume = 0
result['Speed'] = result['Speed']
result['Volume'] = result['Volume'].fillna(0)

#filter out artificial rows (due to cartesian product) where both Speed = NaN & Volume = 0
result = result[~((result['Speed'].isna()) & (result['Volume'] == 0))]

#re-create Full Date col & sort
result['Full Date'] = pd.to_datetime(result[['Year', 'Month', 'Day', 'Hour']])
result = result.sort_values(by=['Intersection Name', 'Full Date', 'Hour'], ascending=[True, False, True])

#rearrange cols
result = result[['Intersection Name', 'Month', 'Day', 'Year', 'Hour', 'Day of Week', 'Direction', 'Speed', 'Volume', 'Full Date']]

#reset index
result = result.reset_index(drop=True)

result.head()

,Intersection Name,Month,Day,Year,Hour,Day of Week,Direction,Speed,Volume,Full Date
0,Azie MortonBarton Springs,7,9,2019,1,2,NB,28.0,9.0,2019-07-09 01:00:00
1,Azie MortonBarton Springs,7,9,2019,1,2,SB,24.0,8.0,2019-07-09 01:00:00
2,Azie MortonBarton Springs,7,9,2019,0,2,NB,28.0,25.0,2019-07-09 00:00:00
3,Azie MortonBarton Springs,7,9,2019,0,2,SB,28.0,36.0,2019-07-09 00:00:00
4,Azie MortonBarton Springs,7,8,2019,23,1,NB,27.0,26.0,2019-07-08 23:00:00


### Adding a Holiday Indicator
We introduce a `Holiday` column to flag whether a date is a recognized holiday. This feature may help analyze how traffic patterns change on holidays compared to regular days.

The holiday dates are pre-defined for each year from 2017 to 2021, covering key national and regional holidays. The holidays used in this analysis are:
1. **New Year's Day** (January 1)  
2. **Valentine's Day** (February 14)  
3. **Easter Sunday** (Varies by year, usually in March or April)  
4. **Mother’s Day** (Second Sunday in May)  
5. **Memorial Day** (Last Monday in May)  
6. **Father’s Day** (Third Sunday in June)  
7. **Independence Day** (July 4)  
8. **Labor Day** (First Monday in September)  
9. **Halloween** (October 31)  
10. **Veterans Day** (November 11)  
11. **Thanksgiving Day** (Fourth Thursday in November)  
12. **Black Friday** (Day after Thanksgiving – unofficial shopping holiday)  
13. **Christmas Day** (December 25)  
14. **New Year's Eve** (December 31)

Each date in the dataset is matched with the corresponding holiday list for its year. The `Holiday` column is then populated as a binary indicator, where:
- **1**: Indicates the date is a holiday.
- **0**: Indicates the date is not a holiday.

In [22]:
#define holidays for each year
holidays = {
    2017: ['01/01', '02/14', '04/16', '05/14', '05/29', '06/18', '04/04', '09/04', '10/31', '11/11', '11/23', '11/24', '12/25', '12/31'],
    2018: ['01/01', '02/14', '04/01', '05/13', '05/28', '06/17', '04/04', '09/03', '10/31', '11/11', '11/22', '11/23', '12/25', '12/31'],
    2019: ['01/01', '02/14', '04/21', '05/12', '05/27', '06/16', '04/04', '09/02', '10/31', '11/11', '11/28', '11/29', '12/25', '12/31'],
    2020: ['01/01', '02/14', '04/12', '05/10', '05/25', '06/21', '04/04', '09/07', '10/31', '11/11', '11/26', '11/27', '12/25', '12/31'],
    2021: ['01/01', '02/14', '04/04', '05/09', '05/31', '06/20', '04/04', '09/06', '10/31', '11/11', '11/25', '11/26', '12/25', '12/31']}

#func to check if a date is a holiday
def is_holiday(row):
    year = row['Year']
    month_day = row['Full Date'].strftime('%m/%d')
    return 1 if year in holidays and month_day in holidays[year] else 0

#'Holiday' col
result['Holiday'] = result.apply(is_holiday, axis=1)

#rearrange columns
result = result[['Intersection Name', 'Month', 'Day', 'Year', 'Hour', 'Day of Week', 'Holiday', 'Direction', 'Speed', 'Volume', 'Full Date']]

result.head()

,Intersection Name,Month,Day,Year,Hour,Day of Week,Holiday,Direction,Speed,Volume,Full Date
0,Azie MortonBarton Springs,7,9,2019,1,2,0,NB,28.0,9.0,2019-07-09 01:00:00
1,Azie MortonBarton Springs,7,9,2019,1,2,0,SB,24.0,8.0,2019-07-09 01:00:00
2,Azie MortonBarton Springs,7,9,2019,0,2,0,NB,28.0,25.0,2019-07-09 00:00:00
3,Azie MortonBarton Springs,7,9,2019,0,2,0,SB,28.0,36.0,2019-07-09 00:00:00
4,Azie MortonBarton Springs,7,8,2019,23,1,0,NB,27.0,26.0,2019-07-08 23:00:00


### Saving the Processed Data
After completing all processing steps, the final dataset is saved as a CSV file (`austin_traffic.csv`). This file will serve as the foundation for exploratory data analysis and subsequent modeling.

In [24]:
result.to_csv('austin_traffic.csv', index=False)

## Exploratory Data Analysis

In [26]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#load data
df = pd.read_csv('austin_traffic.csv')
df.head()

,Intersection Name,Month,Day,Year,Hour,Day of Week,Holiday,Direction,Speed,Volume,Full Date
0,Azie MortonBarton Springs,7,9,2019,1,2,0,NB,28.0,9.0,2019-07-09 01:00:00
1,Azie MortonBarton Springs,7,9,2019,1,2,0,SB,24.0,8.0,2019-07-09 01:00:00
2,Azie MortonBarton Springs,7,9,2019,0,2,0,NB,28.0,25.0,2019-07-09 00:00:00
3,Azie MortonBarton Springs,7,9,2019,0,2,0,SB,28.0,36.0,2019-07-09 00:00:00
4,Azie MortonBarton Springs,7,8,2019,23,1,0,NB,27.0,26.0,2019-07-08 23:00:00


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567532 entries, 0 to 567531
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Intersection Name  567532 non-null  object 
 1   Month              567532 non-null  int64  
 2   Day                567532 non-null  int64  
 3   Year               567532 non-null  int64  
 4   Hour               567532 non-null  int64  
 5   Day of Week        567532 non-null  int64  
 6   Holiday            567532 non-null  int64  
 7   Direction          567532 non-null  object 
 8   Speed              567532 non-null  float64
 9   Volume             567532 non-null  float64
 10  Full Date          567532 non-null  object 
dtypes: float64(2), int64(6), object(3)
memory usage: 47.6+ MB
